In [292]:
import pandas as pd
import numpy as np
import re
import matplotlib as plt
import seaborn as sns
import warnings
import altair as alt
import cpi
%matplotlib inline
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 400)

In [293]:
#Load in HUD data from https://lihtc.huduser.gov/
HUD_lihtc_filepath = 'LIHTCPUB.csv'
hud = pd.read_csv(HUD_lihtc_filepath)

In [294]:
#load in HUD data 
syndicator_data_concat = pd.read_csv("syndicator_data_concat.csv")

#### Some quick checks on year placed in service reliability (can skip to matching sections) 

Below I am checking for differences between year placed in service 
and construction completion date in the syndicator data.

In [295]:
con_complete= syndicator_data_concat[["concomdate","yr_pis","df"]]
con_complete=con_complete.dropna()
con_complete["concomdate"] = "20" + con_complete["concomdate"].str[-2:]
con_complete["diff"] = con_complete["yr_pis"].astype(int) - con_complete["concomdate"].astype(int) 
con_complete["diff_sign"] = np.sign(con_complete["diff"])
con_complete["diff_sign"].value_counts()

 0    3254
-1     320
 1      18
Name: diff_sign, dtype: int64

The above shows that in 320 out of 3592 (8.9 percent) cases we have a construction completion date after a placed in service date. This could be plausible, if a building was started to be rented out before it was totally completed, but could also point to data errors. 

In [296]:
con_complete_no_zeros = con_complete[(con_complete['diff_sign'] != 0)&(con_complete['diff_sign'] != 1)]
con_complete_no_zeros[["df","diff_sign"]].groupby("df").count()

,diff_sign
df,
a,21
c,13
e,9
f,277


The vast majority of these circumstances came from a dataset where placed in service and construction completion date were given to us by the syndicator. As you can note below, the raw data from F contains both columns. 

In [297]:
f = pd.read_csv('datasets/Syndicator F Data Set.csv',skiprows=1)#unaltered Syndicator data 
f.columns

Index(['Deal Id', 'Total Project Cost', 'Total Hard Cost', '# of Sources',
       'Debt Inst Type Name', 'Financing Type', 'Hard vs Soft', 'Amount',
       'LOI Date', 'Close Date', 'Constr Start', 'Constr End', 'PIS',
       'Stabilization Date', 'Total Units', 'City', 'State', 'Zip',
       'Target Population'],
      dtype='object')

As a further robustness test for the PIS data, as it will be key to matching, I want to examine where it falls in relationship to the stabilizaiton date. Theoretically, the placed in service date should always procede the stabilitzation date. 

In [298]:
stab_date_check = syndicator_data_concat[["yr_pis","stabdate","df"]].dropna()
stab_date_check["stabdate"] = "20" + stab_date_check["stabdate"].str[-2:]
stab_date_check["diff"]= stab_date_check["yr_pis"].astype(int) - stab_date_check["stabdate"].astype(int)
stab_date_check["diff_sign"] = np.sign(stab_date_check["diff"])
stab_date_check["diff_sign"].value_counts()

-1    2168
 0    1953
 1     609
Name: diff_sign, dtype: int64

It does not do so 12.8 percent of the time.

In [299]:
stab_date_check[["df","diff_sign"]].groupby("df").describe()

diff_sign                                        
       count  mean  std   min   25%   50%   75%  max
df                                                  
a      79.00 -0.86 0.38 -1.00 -1.00 -1.00 -1.00 1.00
b     275.00 -0.64 0.52 -1.00 -1.00 -1.00  0.00 1.00
c     997.00 -0.65 0.52 -1.00 -1.00 -1.00  0.00 1.00
d    1175.00 -0.29 0.51 -1.00 -1.00  0.00  0.00 1.00
e     448.00 -0.33 0.52 -1.00 -1.00  0.00  0.00 1.00
f     983.00  0.33 0.80 -1.00  0.00  1.00  1.00 1.00
g     154.00 -0.86 0.40 -1.00 -1.00 -1.00 -1.00 1.00
h     306.00 -0.84 0.37 -1.00 -1.00 -1.00 -1.00 0.00
i     227.00 -0.22 0.47 -1.00  0.00  0.00  0.00 1.00
j      86.00 -0.77 0.42 -1.00 -1.00 -1.00 -1.00 0.00

This problem affects every syndicator dataset but J and K.

In [300]:
stab_date_check_no_zeros = stab_date_check[(stab_date_check['diff_sign'] != 0)&(con_complete['diff_sign'] != 1)]
stab_date_check_no_zeros[["df","diff_sign"]].groupby("df").count()

,diff_sign
df,
a,69
b,188
c,689
e,171
f,710
g,139
h,257
i,60


B,C,E,G,H,I were the datasets where construction completion date was substituted for PIS date. These datasets do seem to do somewhat worse on this check (especially c), but notably, F, which has the most discrepancies, is not one.  

#### By showing this I am just trying to get a sense of how skeptical we should be of the year placed in service date to see how much wiggle room we should give matching on year. I will artfully translate this moderate skepticism into "2 years wiggle room". 

In [301]:
#this is just a quick check to see average distance between construction start date and year placed in service. Its ~1.5
#and since we arent using months, for the later analysis rounding up to 2 seems fine

syndicator_data_concat_con = syndicator_data_concat.dropna(subset=["yr_pis","con_stdate"])

def fixdate(x):
    if "/" in x:
        year = x[-2:]
        if year[0] == "9":
            return "19" + year
        else:
            return "20" + year 
    else:
        return x
syndicator_data_concat_con["con_stdate"] = syndicator_data_concat_con.con_stdate.apply(lambda x: fixdate(x))
syndicator_data_concat_con["gap"] = abs(syndicator_data_concat_con["yr_pis"].astype(int) - syndicator_data_concat_con["con_stdate"].astype(float))
syndicator_data_concat_con["gap"].mean()

1.4892672094744634

In [302]:
syndicator_data_concat_con["gap"].describe()

count   4053.00
mean       1.49
std       31.31
min        0.00
25%        1.00
50%        1.00
75%        1.00
max     1994.00
Name: gap, dtype: float64

### Matching 

In [303]:
#take subset of cols we care about
syndicator_data_concat.rename(columns={'df':'syndicator'}, inplace=True)#rename col 
df = syndicator_data_concat[['id', 'syndicator', 'state', 'city', 'zipcode', 'yr_pis',
                             'units_n', 'units_li', 'units_ot', 'sources_n', 'tpop',"totalcost","sources_n","con_stdate"]]

In [304]:
syndicator_data_concat.head()

,Unnamed: 0,id,state,totalcost,hardcost,loi_date,lt_date,yr_pis,concomdate,stabdate,units_n,sources_n,ddf,syndicator,constcost,acqucost,landcost,equity_lif,equity_lis,con_type,lien1,lien1type,lien2,lien2type,lien3,lien3type,lien4,lien4type,lien5,lien5type,lien6,lien6type,lien7,lien7type,lien8,lien8type,lien9,lien9type,lien10,lien10type,acqudate,con_stdate,units_li,units_ot,zipcode,build_n,sqft,tpop,source_type,source_tot,closedate,city,source_cmp,source_pmp,cont_cost,status,fin_type,hard_soft,loan_am,sqft_bldg,sqft_rent,equity_net,source1type,source1,source2type,source2,source3type,source3,source4type,source4,source5type,source5,source6type,source6,source7type,source7,source8type,source8,source9type,source9,source10type,source10,source11type,source11,propname,manager_name
0,0,1224,NY,"$48,299,398.00","$23,352,306.00",11/24/14,09/18/15,2014.00,12/23/14,03/01/15,78.00,0,No,a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,827,NY,"$18,473,998.00","$4,153,972.00",NaN,05/28/09,2009.00,11/19/10,01/19/11,84.00,4,Yes,a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1355,CA,"$24,633,632.00","$12,863,510.00",09/02/16,03/24/17,2018.00,11/20/18,08/05/19,51.00,5,Yes,a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,822,CA,"$9,760,204.00","$1,347,795.00",NaN,09/09/08,2009.00,12/31/09,07/21/10,64.00,1,No,a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1394,CA,"$29,216,019.00","$18,606,578.00",12/14/15,03/01/16,2017.00,12/01/17,10/10/18,60.00,2,Yes,a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [305]:
#create new unique indentifiers for each syndicator 
df.rename(columns={'id':'syn_id'}, inplace=True)
df['id'] = np.arange(df.shape[0])


In [306]:
#subset HUD lihtc data for datasets we might match on plus cols we want 
hudf = hud[['hud_id', 'project', 'proj_cty', 'proj_st', 'proj_zip', 'yr_pis', 'n_units', 'li_units', "type",
"credit","bond"]]

In [307]:
#check on data availability of city and zipcode for syndicator data 
#we can see that we wont be able to match A & G on location. Farther down i add on G b/c it has construction type as a 
#column. A is just left out of this analysis as I don't think its possible to match on, but it also only has 93 entries.
df.groupby('syndicator').apply(lambda x: x.notnull().mean())[['city', 'zipcode']]

,city,zipcode
syndicator,,
a,0.00,0.00
b,0.00,1.00
c,1.00,1.00
d,1.00,1.00
e,1.00,0.00
f,0.42,1.00
g,0.00,0.00
h,1.00,0.00
i,1.00,1.00


In [308]:
#match on zipcode 
zip_match = pd.merge(df.dropna(subset=['zipcode']), hudf.dropna(subset=['proj_zip']), left_on='zipcode', right_on='proj_zip')

In [309]:
#do some cleaning of city name 
hudf.loc[hudf.proj_cty.notnull(), 'city'] = hudf.loc[hudf.proj_cty.notnull(), 'proj_cty'].apply(lambda x: x.lower())
df.loc[df.city.notnull(), 'city'] = df.loc[df.city.notnull(), 'city'].apply(lambda x: x.lower())

In [310]:
#match on city 
city_match = pd.merge(df.dropna(subset=['city']), hudf.dropna(subset=['city']), left_on=['city', 'state'],
                     right_on=['city', 'proj_st'])

In [311]:
# yr_pis match within two year for both city and zip group 
zip_match_yr = zip_match[abs(zip_match.yr_pis_x - zip_match.yr_pis_y)<=2]
city_match_yr = city_match[abs(city_match.yr_pis_x - city_match.yr_pis_y)<=2]

The gap in the whole of the HUD data between the number of lihtc units and total units is about 5 percent. Which seems 
as good as a margin of error as any to use for number of units. 

In [312]:
#filter by +/-5% number of units 
zip_match_yr = zip_match_yr[(abs(zip_match_yr.units_n/zip_match_yr.n_units) >= .95)&(abs(zip_match_yr.units_n/zip_match_yr.n_units) <= 1.05)] 
city_match_yr = city_match_yr[(abs(city_match_yr.units_n/city_match_yr.n_units) >= .95)&(abs(city_match_yr.units_n/city_match_yr.n_units) <= 1.05)] 

In [313]:
#label type of match 
matches = zip_match_yr[~zip_match_yr.id.duplicated()]#[['id', 'hud_id']]
matches['match_type'] = 'zip+yr'

c1 = city_match_yr[~city_match_yr.id.duplicated()]#[['id', 'hud_id']]
c1['match_type'] = 'city+yr'

In [314]:
#add both types of matches together 
matches = matches.append(c1[~(c1.id.isin(matches.id))])

In [315]:
matches = matches.loc[:,~matches.columns.duplicated()] #double check no duplicate cols before putting to csv


In [258]:
matches.head()

,syn_id,syndicator,state,city,zipcode,yr_pis_x,units_n,units_li,units_ot,sources_n,tpop,totalcost,sources_n,con_stdate,id,hud_id,project,proj_cty,proj_st,proj_zip,yr_pis_y,n_units,li_units,type,credit,bond,match_type
10,33663,b,ME,NaN,04426,2015.00,24.00,24.00,0.00,3.0,Elderly,"$5,333,498",3.0,11/1/14,94,MEA20150006,NORTH VIEW APARTMENTS,DOVER FOXCROFT,ME,04426,2015,24.00,24.00,1.00,1.00,1.00,zip+yr
11,33709,b,OK,NaN,73134,2013.00,44.00,44.00,0.00,2.0,Elderly,"$6,849,238",2.0,9/21/12,95,OKA20130004,CROSS CREEK LANDING,OKLAHOMA CITY,OK,73134,2013,44.00,nan,1.00,2.00,2.00,zip+yr
13,33768,b,NC,NaN,27896,2015.00,72.00,72.00,0.00,3.0,Family,"$7,183,151",3.0,2/20/14,96,NCA20150020,RAVENSWOOD APARTMENTS,WILSON,NC,27896,2015,72.00,72.00,1.00,2.00,2.00,zip+yr
25,34280,b,NJ,NaN,07208,2014.00,84.00,83.00,1.00,4.0,Family,"$20,987,345",4.0,7/18/13,98,NJA20120412,WESTMINSTER HEIGHTS,ELIZABETH,NJ,07208,2012,84.00,83.00,1.00,nan,2.00,zip+yr
33,34320,b,NC,NaN,28412,2015.00,60.00,60.00,0.00,5.0,Family,"$6,965,402",5.0,3/28/14,99,NCA20150016,LOCKWOOD VILLAGE APARTMENTS,WILMINGTON,NC,28412,2015,60.00,60.00,1.00,2.00,2.00,zip+yr


In [259]:
#keep year construction starts and clean it if it exists, if it does no exist us PIS - 2 

#clean up construction start year
df["con_stdate"] = df["con_stdate"].astype(str)
matches["con_stdate_PIS"] = matches["yr_pis_x"] - 2 
matches.loc[matches.con_stdate.notnull(), 'con_stdate'] = matches.loc[matches.con_stdate.notnull(), 'con_stdate'].apply(lambda x: fixdate(x))
matches.loc[matches.con_stdate.isnull(), 'con_stdate'] = matches.loc[matches.con_stdate.isnull(), 'con_stdate_PIS']#.apply(lambda x: sjdfkjdkfj)
matches = matches.rename(columns={"con_stdate":"Year"})

In [260]:
#_x is syndicator data 
matches = matches.rename(columns={"yr_pis_x":"yr_pis"})

In [316]:
#matches.to_csv("syndicator_pre_drop_v3.csv")

In [264]:
#want to filter for credit == 2, which means a 70% subsidy, aka 9& percent credit project. 
matches = matches[matches.credit == 2]

In [265]:
# there are 733 nulls, which we are throwing out entirely 
matches.credit.isna().sum()

0

In [266]:
# ~30 percent of these projects are not new constufction 
matches.type.value_counts()

1.00    1090
2.00     277
3.00      31
Name: type, dtype: int64

In [267]:
#this is code ot read in data from syndicator G, leaving in case we decide to do so later 
#As mentioned above, G does not have enough location data to use, but I will filter for new construction and append to our dataset
#read in data (these are the syndicator data post Carson's initial pre-processing)
#g = pd.read_csv("/Users/quinnunderriner/Desktop/Work/syndictor_lihtc/syndicator_g.csv")
#g = g[g.con_type == "New Construction"]
#g = g.rename(columns={"df":"syndicator"})
#append new construction G onto matches
#matches = matches.append(g)

### Quick analysis and graphing 

In [270]:
#this sets color and font theme for altair 
def terner_theme2():
    # Typography
    font = "Veranda Regular"
    labelFont = "Veranda Regular" 
    sourceFont = "Veranda Regular"
    # Axes
    axisColor = "#FFFFFF"
    gridColor = "#DEDDDD"
    markColor = "#a6833d"
    # Colors
    main_palette = ["#FFB81D",
                   "#4E748B",
                   "#B7B09D"]
                    
    sequential_palette = ["#FFB81D",
                   "#4E748B",
                   "#B7B09D"]
    
    return{"width": 400, 
          "height": 400, 
          "config": {
              "title": {
                  "fontSize": 18,
                  "font": font,
                  "anchor": "start", 
                  "fontColor": "#000000"
              },
              
              "range": {
                  "category": main_palette,
                  "diverging": sequential_palette,

              
            },
                      "legend": {
                  "labelFont": labelFont,
                  "labelLimit": 0
                      },
}
    }

# register
alt.themes.register("terner_theme2", terner_theme2)
# enable
alt.themes.enable("terner_theme2")

ThemeRegistry.enable('terner_theme2')

In [271]:
def adjust_for_CPI(date,house_cost):
    if date != 2020:
        return cpi.inflate(house_cost, date)
    else:
        return house_cost

def clean_up_for_charts(df):
    #make yr_pis an int, and substract two years for con_yr per discussion 
    #df["Year"] = df["yr_pis_y"].astype(int) - 2
    #df = df.drop(columns={"yr_pis"})
    df.Year = df.Year[(df.Year != 8886)&(df.Year != 9997)]

    df = df.dropna(subset=["totalcost","Year","units_n"])
    df['totalcost'] = df['totalcost'].replace({'\$': '', ',': ''}, regex=True)
    df['totalcost'] = df['totalcost'].astype(int)
    #find average cost per unit, adjusted for cpi 
    
    df["Year"] = df["Year"].astype(int)
    df['totalcost_adj'] = df[["Year","totalcost"]].apply(lambda x: adjust_for_CPI(*x), axis=1)
    
    df["Cost Per Unit"] = df["totalcost_adj"] / df["units_n"].astype(int)
    df = df.dropna(subset=["Cost Per Unit"])

    #clean up name 
    df = df.rename(columns = {"sources_n":"Number of Sources","tpop":"Target Population"})
    df["Number of Sources"] = df["Number of Sources"].astype(float)
    return df

In [272]:
matches = clean_up_for_charts(matches)
matches = matches.loc[:,~matches.columns.duplicated()] #double check no duplicate cols before putting to csv

In [273]:
matches["Target Population"] = matches["Target Population"].replace(
    {"Senior - Age Restricted": 'Senior',
    'Elderly':"Senior",
    "Special Needs":"Supportive Housing",
    "Senior (62+)":"Senior",
    "Senior (55+)":"Senior",
    "Formerly Homeless":"Supportive Housing"})

In [274]:
#dropped mixed 
matches["Target Population"].value_counts()

Family                     957
Senior                     354
Supportive Housing         117
Mixed - Family & Senior      5
SRO                          3
Native American              2
Family / Senior              2
Mixed                        2
Homeless                     1
Veterans                     1
Name: Target Population, dtype: int64

In [275]:
matches = matches.loc[:,~matches.columns.duplicated()] #double check no duplicate cols before putting to csv
#matches.to_csv("big_syndicator_data_set.csv")

In [281]:
#need to group by year to be able to make charts 
grouped_match = matches.groupby(["Year"]).mean().reset_index()
#need to sort by year and make it a string to not have a comma (like 2,004) in the year name
grouped_match = grouped_match.sort_values("Year")
grouped_match.Year = grouped_match.Year.astype(str)
#make sure no duplicate cols for altair
grouped_match = grouped_match.loc[:,~grouped_match.columns.duplicated()]


In [282]:
alt.Chart(grouped_match).mark_line().encode(
    x='Year',
    y='Cost Per Unit').configure(background="#ffffff").configure_legend().properties(
    title={
      "text": ["Syndicator Cost Per Unit Over Time (CPI Adjusted 2020)"]})


alt.Chart(...)

In [283]:
grouped_match = grouped_match[grouped_match.Year >= "1991"]
alt.Chart(grouped_match).mark_line().encode(
    x='Year',
    y='Number of Sources'
).configure(background="#ffffff").configure_legend().properties(
    title={
      "text": ["Syndicator Projects: Number of Sources Over Time"]})

alt.Chart(...)

In [284]:
matches_target_pop = matches
matches_target_pop["Target Population"] = matches_target_pop["Target Population"].replace(
    {"Senior - Age Restricted": 'Senior',
    'Elderly':"Senior",
    "Special Needs":"Supportive Housing",
    "Senior (62+)":"Senior",
    "Senior (55+)":"Senior",
    "Formerly Homeless":"Supportive Housing"})

In [285]:
matches_target_pop["Target Population"].value_counts()

Family                     957
Senior                     354
Supportive Housing         117
Mixed - Family & Senior      5
SRO                          3
Native American              2
Family / Senior              2
Mixed                        2
Homeless                     1
Veterans                     1
Name: Target Population, dtype: int64

In [286]:
matches_target_pop = matches_target_pop[matches_target_pop["Target Population"].isin(["Senior","Family","Supportive Housing"])]
#need to sort by year and make it a string to not have a comma (like 2,004) in the year name
matches_target_pop = matches_target_pop.sort_values("Year")
matches_target_pop.Year = matches_target_pop.Year.astype(str)
matches_target_pop=matches_target_pop.groupby(["Year","Target Population"]).mean().reset_index()

In [287]:
alt.Chart(matches_target_pop).mark_line().encode(
    x='Year',
    y='Cost Per Unit',
    color="Target Population"
).configure(background="#ffffff").configure_legend().properties(
    title={
      "text": ["Syndicator Cost Per Unit Over Time by Target Population"]})

alt.Chart(...)

In [288]:
matches_target_pop = matches_target_pop[matches_target_pop.Year > "1997"]
alt.Chart(matches_target_pop).mark_line().encode(
    x='Year',
    y='Number of Sources',
    color="Target Population"
).configure(background="#ffffff").configure_legend().properties(
    title={
      "text": ["Syndicator Number of Sources Over Time by Target Population"]})

alt.Chart(...)

In [289]:
matches_target_pop = matches_target_pop.groupby("Target Population").mean().reset_index()

In [290]:
alt.Chart(matches_target_pop).mark_bar().encode(
    x='Target Population',
    y='Number of Sources',
    color="Target Population")

alt.Chart(...)